In [1]:
import scipy.io as sp
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import accuracy_score
from functions import *

In [18]:
data = sp.loadmat('S1.mat')
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'sub', 'Fs', 'N', 'T', 'exemplarLabels', 'categoryLabels', 'X_2D', 'X_3D'])


In [19]:
print(data['exemplarLabels'].shape)

(1, 5188)


In [20]:
lab = data['categoryLabels'].transpose()
lab = np.reshape(lab, (lab.shape[0]*lab.shape[1]))
print(lab)

[4 6 3 ... 5 1 1]


In [21]:
matrix = data['X_3D'].transpose(2,0,1)

In [22]:
electrode = []
cat_labels = []
for i in range(5188):
    if(lab[i]==2):
        electrode.append(matrix[i])
        cat_labels.append(1)
    elif(lab[i]==6):
        electrode.append(matrix[i])
        cat_labels.append(2)
electrode = np.array(electrode)
cat_labels = np.array(cat_labels)
print(electrode.shape)
print(cat_labels.shape)

(1728, 124, 32)
(1728,)


In [23]:
old_matrix = matrix
matrix = electrode

In [24]:
matrix = np.reshape(matrix,(matrix.shape[0], matrix.shape[1]*matrix.shape[2]))
matrix = preprocessing.scale(matrix)

In [25]:
matrix.shape

(1728, 3968)

In [26]:
print(cat_labels.shape)

(1728,)


In [29]:
np.max(cat_labels)

2

In [17]:
acc = cross_validation(matrix, cat_labels)
print(acc)

0.7424855491329481


In [15]:
def split_sampling(matrix, cat_labels, k = 0.8):
    n = matrix.shape[0]
    indices = np.random.permutation(n)
    training_idx, test_idx = indices[:int(k*n)], indices[int(k*n):]
    #print(int(k*n))
    x_train, x_test = matrix[training_idx], matrix[test_idx]
    y_train, y_test = cat_labels[training_idx], cat_labels[test_idx]
    #print(indices)
    x_train = np.reshape(x_train, newshape = (x_train.shape[0],1))
    x_test = np.reshape(x_test, newshape = (x_test.shape[0],1))
    return x_train, y_train, x_test, y_test

In [16]:
def split_sampling2D(matrix, cat_labels, k = 0.8):
    n = matrix.shape[0]
    indices = np.random.permutation(n)
    training_idx, test_idx = indices[:int(k*n)], indices[int(k*n):]
    #print(int(k*n))
    x_train, x_test = matrix[training_idx,:], matrix[test_idx,:]
    y_train, y_test = cat_labels[training_idx], cat_labels[test_idx]
    #print(indices)
    return x_train, y_train, x_test, y_test

In [16]:
def lda_with_shrinkage(x_train, y_train, x_test, y_test):
    clf = LDA(solver = 'eigen',shrinkage = True)
    clf.fit(x_train, y_train)
    pred = clf.predict(x_test)
    return accuracy_score(pred,y_test)

In [14]:
def cross_validation(matrix, cat_labels, k = 0.8, iterations = 10):
    accuracy = []
    for i in range(iterations):
        if(len(matrix.shape)>1):
            x_train, y_train, x_test, y_test = split_sampling2D(matrix, cat_labels)
        else:
            x_train, y_train, x_test, y_test = split_sampling(matrix, cat_labels)
        acc = lda_with_shrinkage(x_train, y_train,x_test, y_test)
        accuracy.append(acc)
    return np.mean(accuracy)

In [ ]:
linear_clf = svm.SVC(kernel = 'linear',verbose = True)
x_train, y_train, x_test, y_test = split_sampling2D(matrix, cat_labels)
linear_clf.fit(x_train, y_train)
linear_svm_pred = linear_clf.predict(x_test)
accuracy_score(linear_svm_pred,y_test)

In [ ]:
electrode = electrode.transpose(1,2,0)
sh0 = electrode.shape[0];
sh1 = electrode.shape[1];
sh2 = electrode.shape[2];
acc = [];
for l in range(sh0):
    temp = []
    for k in range(sh1):
        x_tr = []; y_tr = [];
        for i in range(sh2):
            x_tr.append(electrode[l][k][i]);
            y_tr.append(cat_labels[i]);
        x_tr = np.array(x_tr)
        y_tr = np.array(y_tr)
        # use x_tr & y_tr to get accuracy and store it in acc_score
        #x_train, y_train, x_test, y_test = split_sampling(x_tr, y_tr)
        
        #print(str(x_train.shape)+" "+str(y_train.shape) )
        #print(str(x_tr.shape)+" " +str(l)+ " " + str(k))
        #temp.append(lda_with_shrinkage(x_train, y_train,x_test, y_test));
        temp.append(cross_validation(x_tr,y_tr))
    acc.append(temp);
# you have acc as 2D matrix (of size 124*32) with every value denoting accuracy accordingly.
# Plot graph of accuracy.

In [ ]:
print(acc)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(acc)

In [ ]:
sh0 = electrode.shape[0];
sh1 = electrode.shape[1];
sh2 = electrode.shape[2];
acc = [];
for l in range(sh0):
    x_tr = []; y_tr = [];
    for i in range(sh2):
        temp = [];
        for k in range(sh1):
            temp.append(electrode[l][k][i]);
        x_tr.append(temp);
        y_tr.append(cat_labels[i])
    x_tr = np.array(x_tr)
    y_tr = np.array(y_tr)
    # use x_tr & y_tr to get accuracy and store it in acc_score
    #x_train, y_train, x_test, y_test = split_sampling2D(x_tr, y_tr)
    acc_score = cross_validation(x_tr, y_tr)#lda_with_shrinkage(x_train, y_train, x_test, y_test)
    acc.append(acc_score);
# you have acc as 2D matrix (of size 124) with every value denoting accuracy accordingly.
# Plot graph of accuracy.

In [ ]:
plt.hist(acc, bins = 124)

In [ ]:
print(acc)

In [ ]:
sh0 = electrode.shape[0];
sh1 = electrode.shape[1];
sh2 = electrode.shape[2];
acc = [];
for k in range(sh1):
    x_tr = []; y_tr = [];
    for i in range(sh2):
        temp = [];
        for l in range(sh0):
            temp.append(electrode[l][k][i]);
        x_tr.append(temp);
        y_tr.append(cat_labels[i]);
    x_tr = np.array(x_tr)
    y_tr = np.array(y_tr)
    # use x_tr & y_tr to get accuracy and store it in acc_score
    #x_train, y_train, x_test, y_test = split_sampling2D(x_tr, y_tr)
    acc_score = cross_validation(matrix = x_tr, cat_labels = y_tr) #lda_with_shrinkage(x_train, y_train, x_test, y_test)
    # use x_tr & y_tr to get accuracy and store it in acc_score
    acc.append(acc_score);
# you have acc as 2D matrix (of size 32) with every value denoting accuracy accordingly.
# Plot graph of accuracy.


In [ ]:
print(acc)
plt.hist(acc, bins = 32)

In [47]:
count = 0;
for i in range(len(cat_labels)):
    if(cat_labels[i]==1):
        count+=1

In [49]:
cat_labels.shape

(1728,)

In [ ]:
plt.plot(acc)

In [56]:
class_wise_xtrain = separate_by_class(matrix , cat_labels)

In [14]:
print(np.array(class_wise_xtrain).shape)

(5,)


In [30]:
augmented_xtrain, augmented_ytrain, augmented_xtest, augmented_ytest = augment(matrix, cat_labels, with_replacement=False)

In [31]:
print(augmented_xtrain.shape,
augmented_ytrain.shape,
augmented_xtest.shape,
augmented_ytest.shape)

(344, 3968) (344,) (86, 3968) (86,)


In [58]:
class_wise_xtrain[0].shape

(864, 3968)

In [59]:
class_wise_xtrain[1].shape

(864, 3968)

In [32]:
acc1 = cross_validation_with_augmentation(matrix,cat_labels,sample_size = 2, with_replacement = True, k = 0.6, niter = 10)

In [33]:
acc1

0.8291907514450868

In [34]:
sam_sizes = [2,4,6,8]
acc_mat = []
for size in sam_sizes:
    acc1 = cross_validation_with_augmentation(matrix,cat_labels,sample_size = size, with_replacement = True, k = 0.8, niter = 10)
    acc2 = cross_validation_with_augmentation(matrix,cat_labels,sample_size = size, with_replacement = False, k = 0.8, niter = 10)
    acc_mat.append([acc1,acc2])

In [35]:
acc_mat

[[0.8286127167630057, 0.8211755612313484],
 [0.9193641618497109, 0.8858002735978111],
 [0.9549132947976879, 0.9368421052631579],
 [0.9641618497109826, 0.9601328903654485]]